# CNG 562 - HOMEWORK 1

# Question 1  : Adult Census Income


In [0]:
#importing json file which is downloaded from kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"burakkizilkaya","key":"ee6f9dc9caf9bfd6f6082598e57c9f44"}'}

In [0]:
#if there is no kaggle directory, create a new one.
!mkdir -p ~/.kaggle
#copy kaggle.json file to kaggle directory
!cp kaggle.json ~/.kaggle/
#give owner read and write access permissions
!chmod 600 ~/.kaggle/kaggle.json
#check files in kaggle directory
!ls ~/.kaggle

kaggle.json


In [0]:
#install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

In [0]:
#download adult-census-income dataset
!kaggle datasets download -d uciml/adult-census-income

adult-census-income.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#unzip downloaded dataset file
!unzip adult-census-income.zip

Archive:  adult-census-income.zip
replace adult.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: adult.csv               


In [0]:
#import data from .csv file
import pandas as pd
import numpy as np
data = pd.read_csv("adult.csv")
#data has ? symbol instead of NaN. Change them to NaN
data = data.replace('?', np.nan)

In [0]:
#Replace NaN values with mean for numeric columns
for column in ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']:
    data[column].fillna(data[column].mean(),inplace=True)


In [0]:
#Replace NaN values with mode for character columns
for column in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'income']:
    data[column].fillna(data[column].mode(),inplace=True)
    
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [0]:
#creating dummy variables for categorical columns
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'income'], drop_first=True)

In [0]:
#normalizing data
tempx = np.array(data.drop('income_>50K', axis=1))
tempy = np.array(data['income_>50K'])
x = (tempx - tempx.min(0)) / tempx.ptp(0)
y = (tempy - tempy.min(0)) / tempy.ptp(0)
print(x)
print(y)

[[1.         0.04398745 0.53333333 ... 1.         0.         0.        ]
 [0.89041096 0.08189579 0.53333333 ... 1.         0.         0.        ]
 [0.67123288 0.11802067 0.6        ... 1.         0.         0.        ]
 ...
 [0.31506849 0.09650032 0.53333333 ... 1.         0.         0.        ]
 [0.56164384 0.09482688 0.53333333 ... 1.         0.         0.        ]
 [0.06849315 0.12849934 0.53333333 ... 1.         0.         0.        ]]
[0. 0. 0. ... 1. 0. 0.]


## Logistic Regression and Grid Search

In [0]:
#split test and train data
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=0)

#logistic regression with sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(xtrain, ytrain)
ypred = logreg.predict(xtest)

print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.8476845596364083
Precision: 0.7298701298701299
Recall: 0.5770020533880903


In [0]:
#grid search for logistic regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
tuned_parameters = [{'solver': ['liblinear'], 'C': [1, 0.75, 0.5, 0.25, 0.1, 0.01], 'penalty': ['l1', 'l2']}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

Grid scores on development set:

0.809 (+/-0.013) for {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.805 (+/-0.013) for {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
0.809 (+/-0.013) for {'C': 0.75, 'penalty': 'l1', 'solver': 'liblinear'}
0.803 (+/-0.014) for {'C': 0.75, 'penalty': 'l2', 'solver': 'liblinear'}
0.808 (+/-0.014) for {'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'}
0.800 (+/-0.016) for {'C': 0.5, 'penalty': 'l2', 'solver': 'liblinear'}
0.808 (+/-0.015) for {'C': 0.25, 'penalty': 'l1', 'solver': 'liblinear'}
0.797 (+/-0.019) for {'C': 0.25, 'penalty': 'l2', 'solver': 'liblinear'}
0.807 (+/-0.015) for {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
0.793 (+/-0.017) for {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.775 (+/-0.017) for {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
0.785 (+/-0.015) for {'C': 0.01, 'pe

## Decision Tree and Grid Search

In [0]:
#decision tree with sklearn 
from sklearn.tree import DecisionTreeClassifier  
 
dt = DecisionTreeClassifier()   
dt.fit(xtrain, ytrain) 

ypred = dt.predict(xtest)
print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

Accuracy: 0.8123080702616386
Precision: 0.6074718526100307
Recall: 0.6093429158110883


In [0]:
#grid search for decision tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
tuned_parameters = [{'criterion': ['gini', 'entropy'], 'max_features' : ['auto', 'log2', 1.0, 8, 0.66], 'min_samples_leaf' : [1, 2, 3]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 3}

Grid scores on development set:

0.737 (+/-0.011) for {'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 1}
0.777 (+/-0.012) for {'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 2}
0.788 (+/-0.024) for {'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 3}
0.728 (+/-0.010) for {'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1}
0.783 (+/-0.020) for {'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 2}
0.786 (+/-0.034) for {'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 3}
0.749 (+/-0.017) for {'criterion': 'gini', 'max_features': 1.0, 'min_samples_leaf': 1}
0.760 (+/-0.016) for {'criterion': 'gini', 'max_features': 1.0, 'min_samples_leaf': 2}
0.761 (+/-0.022) for {'criterion': 'gini', 'max_features': 1.0, 'min_samples_leaf': 3}
0.733

## Support Vector Machines and Grid Search

In [0]:
#support vector machine with sklearn
from sklearn import svm

supp = svm.SVC(gamma='scale')
supp.fit(xtrain, ytrain)
ypred = supp.predict(xtest)

print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

Accuracy: 0.8392089423903697
Precision: 0.7274021352313167
Recall: 0.5246406570841889


In [0]:
#grid search for support vector machine
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import svm

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()

# Tuning hyper-parameters for precision



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classificatio

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.795 (+/-0.023) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.379 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.791 (+/-0.018) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.795 (+/-0.022) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.800 (+/-0.016) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.791 (+/-0.019) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.813 (+/-0.012) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.801 (+/-0.015) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.808 (+/-0.018) for {'C': 1, 'kernel': 'linear'}
0.809 (+/-0.014) for {'C': 10, 'kernel': 'linear'}
0.809 (+/-0.014) for {'C': 100, 'kernel': 'linear'}
0.809 (+/-0.014) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

           

## K-Nearest Neighbour and Grid Search

In [0]:
#knn with sklearn
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier()
neigh.fit(xtrain, ytrain) 
ypred = neigh.predict(xtest)

print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

Accuracy: 0.8248372435818695
Precision: 0.6524532710280374
Recall: 0.5734086242299795


In [0]:
#grid search for support vector machine
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import svm

# Set the parameters by cross-validation
tuned_parameters = [{'n_neighbors': [3, 4, 5, 6, 7, 8, 9], 'weights' : ['uniform', 'distance']}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()